In [ ]:
#エッジ用のnumpy保存プログラム 2023年度用
import numpy as np
from PIL import Image
import cv2
import os,glob,random

outfile="./num16karnel3light32px.npz"#変更必要
max_photo=40000
photo_size=32#変更必要
x=[]
y=[]
laplace5=[[-1,-3,-4,-3,-1],
         [-3,0,6,0,-3],
         [-4,6,20,6,-4],
         [-3,0,6,0,-3],
         [-1,-3,-4,-3,-1]]
laplace3=[[1,1,1],
         [1,-8,1],
         [1,1,1]]

def main():
    glob_files("./2023grayhands/aizawa/left/omote",0)
    glob_files("./2023grayhands/aizawa/left/ura",1)
    glob_files("./2023grayhands/aizawa/right/omote",2)
    glob_files("./2023grayhands/aizawa/right/ura",3)
    glob_files("./2023grayhands/arai/left/omote",4)
    glob_files("./2023grayhands/arai/left/ura",5)
    glob_files("./2023grayhands/arai/right/omote",6)
    glob_files("./2023grayhands/arai/right/ura",7)
    glob_files("./2023grayhands/ebato/left/omote",8)
    glob_files("./2023grayhands/ebato/left/ura",9)
    glob_files("./2023grayhands/ebato/right/omote",10)
    glob_files("./2023grayhands/ebato/right/ura",11)
    glob_files("./2023grayhands/isii/left/omote",12)
    glob_files("./2023grayhands/isii/left/ura",13)
    glob_files("./2023grayhands/isii/right/omote",14)
    glob_files("./2023grayhands/isii/right/ura",15)
    glob_files("./2023grayhands/saitouryouga/left/omote",16)
    glob_files("./2023grayhands/saitouryouga/left/ura",17)
    glob_files("./2023grayhands/saitouryouga/right/omote",18)
    glob_files("./2023grayhands/saitouryouga/right/ura",19)
    glob_files("./2023grayhands/sakuma/left/omote",20)
    glob_files("./2023grayhands/sakuma/left/ura",21)
    glob_files("./2023grayhands/sakuma/right/omote",22)
    glob_files("./2023grayhands/sakuma/right/ura",23)
    glob_files("./2023grayhands/tanaka/left/omote",24)
    glob_files("./2023grayhands/tanaka/left/ura",25)
    glob_files("./2023grayhands/tanaka/right/omote",26)
    glob_files("./2023grayhands/tanaka/right/ura",27)
    glob_files("./2023grayhands/tokuyama/left/omote",28)
    glob_files("./2023grayhands/tokuyama/left/ura",29)  
    glob_files("./2023grayhands/tokuyama/right/omote",30)
    glob_files("./2023grayhands/tokuyama/right/ura",31)
    glob_files("./2023grayhands/watanabe/left/omote",32)
    glob_files("./2023grayhands/watanabe/left/ura",33)
    glob_files("./2023grayhands/watanabe/right/omote",34)
    glob_files("./2023grayhands/watanabe/right/ura",35)
    glob_files("./2023grayhands/yamaguchi/left/omote",36)
    glob_files("./2023grayhands/yamaguchi/left/ura",37)
    glob_files("./2023grayhands/yamaguchi/right/omote",38)
    glob_files("./2023grayhands/yamaguchi/right/ura",39)
    np.savez(outfile,x=x,y=y)
    print("保存しました:"+outfile,len(x))
    print(x[0],y[0])
    
def glob_files(path,label):
    files=glob.glob(path+"/*.jpg")
    random.shuffle(files)
    num=0
    for f in files:
        if num>=max_photo: break
        num+=1
        
        #---------edgesの時のプログラム--------------
        img=cv2.imread(f)
        img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        karnel=np.array(laplace3,np.float64)#変更必要
        edges=cv2.filter2D(img_gray,-1,karnel)
        edges=cv2.resize(edges,(photo_size,photo_size))
        
        
        edges=np.asarray(edges)#1つの配列に格納（この場合2つ）
        x.append(edges)
        y.append(label)
if __name__=='__main__':
    main()

In [ ]:
%%time
#エッジ検出を用いた機械学習
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    model.summary()
    return model

def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model


im_rows = 256 # 変更必要
im_cols = 256 # 変更必要
im_color = 1
in_shape = (im_rows, im_cols, im_color)
nb_classes = 16 #変更必要

photos = np.load('./num16karnel5light256px.npz')#変更必要
x = photos['x']
y = photos['y']

x = x.reshape(-1, im_rows, im_cols, im_color)
x = x.astype('float32') / 255
y = keras.utils.to_categorical(y.astype('int32'), nb_classes)

x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size=0.9)

model = get_model(in_shape, nb_classes)


hist = model.fit(x_train, y_train,
          batch_size=256,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('正解率=', score[1], 'loss=', score[0])


plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

model.save_weights('./hdf/num16karnel5light256pxbatch256.hdf5') #変更必要

In [ ]:
#他人受入率の評価プログラム
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image


def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    return model


def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model


#他人受入率用のテスト配列
omote=["saitouhide/omote","saitouryou/omote",
      "sakuma/omote","tanaka/omote","tokuyama/omote","watanabe/omote",
      "yamaguchi/omote"]
ura=["saitouhide/ura","saitouryou/ura",
      "sakuma/ura","tanaka/ura","tokuyama/ura","watanabe/ura",
      "yamaguchi/ura"]


target_omote=[]
target_ura=[]
laplace5=[[-1,-3,-4,-3,-1],
         [-3,0,6,0,-3],
         [-4,6,20,6,-4],
         [-3,0,6,0,-3],
         [-1,-3,-4,-3,-1]]
laplace3=[[1,1,1],
         [1,-8,1],
         [1,1,1]]
passpercent=[]
notpasspercent=[]
omotepercent=[]
urapercent=[]
for k in range(7):#変更必要
    for i in range(10):
        target_omote.append("./graytest/"+str(omote[k])+"/"+str(i)+".jpg")#変更必要
        target_ura.append("./graytest/"+str(ura[k])+"/"+str(i)+".jpg")#変更必要


im_rows = 64 #変更必要
im_cols = 64 #変更必要
im_color = 1 #変更必要
in_shape = (im_rows, im_cols, im_color)
nb_classes = 16 #変更必要
#配列の最大値　11月追加
max_score = 0.0

#他人受入率用のラベル

LABELS=["江波戸の表","江波戸の裏","奥谷君の表","奥谷君の裏",
       "栃窪先生の表","栃窪先生の裏","中村君の表","中村君の裏",
       "後藤さんの表","後藤さんの裏","三村君の表","三村君の裏",
       "新井さんの表","新井さんの裏","石井さんの表","石井さんの裏"]

model = get_model(in_shape, nb_classes)
model.load_weights('./hdf/11-3min/num16karnel5light64pxbatch256.hdf5')#変更必要

#最大値を見つける関数 11月追加
def findmaxscore(per):
    global max_score
    if per!=100.0:
        if max_score<=per:
            max_score=per 

def check_photo(path):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    karnel=np.array(laplace5,np.float64)#変更必要
    img=cv2.filter2D(img,-1,karnel)
    img=cv2.resize(img,(im_rows,im_cols))
    plt.imshow(img,cmap="gray")
    plt.show()

    x=np.asarray(img)
    x=x.reshape(-1,im_rows,im_cols,im_color)
    x=x/255
    
    #予測
    pre=model.predict([x])[0]#画像の予測
    idx=pre.argmax()
    per=(pre[idx]*100)
    if per<99.999 and ("omote" in path): #閾値変更箇所
        omotepercent.append(per)
    elif per<99.999 and ("ura" in path): #閾値変更箇所
        urapercent.append(per)
    else:
        pass
        
        
    return(idx,per)

def check_photo_str(path):
    idx,per=check_photo(path)
    if per>=99.999: #閾値変更箇所
        print("この写真は、",LABELS[idx],"です。")
        print("可能性は、",per,"%です。")
        passpercent.append(per)
        #配列の最大値を見つける　11月追加
        findmaxscore(per)
    else:
        print('\033[31m'+"可能性が",per,"%なので認証できません。"+'\033[31m')
        print("認証の結果は",LABELS[idx],"でした。")
        notpasspercent.append(per)
    
if __name__=='__main__':
    number=[9,19,29,39,49,59,69,79,89,99,109,119,129,139,149]
    for i in range(70):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_omote[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
    for i in range(70):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_ura[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
    print(passpercent)
    print(notpasspercent)
    print("閾値以上で識別したテスト数"+str(len(passpercent)))
    print("閾値未満で識別したテスト数"+str(len(notpasspercent)))
    print("表の手だけで閾値未満で識別したテスト数"+str(len(omotepercent)))
    print("裏の手だけで閾値未満で識別したテスト数"+str(len(urapercent)))
    totalpass=len(passpercent)/140
    totalnotpass=len(notpasspercent)/140
    omotenotpass=len(omotepercent)/70
    omotepass=(70-len(omotepercent))/70
    uranotpass=len(urapercent)/70
    urapass=(70-len(urapercent))/70
    print("================================================")
    print("未学習の手を閾値未満で識別した割合"+str(round(totalnotpass*100))+"%")
    print("未学習の手を閾値以上で識別した割合"+str(round(totalpass*100))+"%")
    print("表の手だけで未学習の手を閾値未満で識別した割合"+str(round(omotenotpass*100))+"%")
    print("表の手だけで未学習の手を閾値以上で識別した割合"+str(round(omotepass*100))+"%")
    print("裏の手だけで未学習の手を閾値未満で識別した割合"+str(round(uranotpass*100))+"%")
    print("裏の手だけで未学習の手を閾値以上で識別した割合"+str(round(urapass*100))+"%")
    print("閾値以上の最大値は"+str(max_score)) #11月追加
    print("閾値以上の最小値は"+str(min(passpercent))) #11月追加

In [ ]:
#本人拒否率の評価プログラム
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image


def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    return model


def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model


#本人受入率用のテスト配列
omote=["mine/omote","okutani/omote","tochikubo/omote",
       "nakamura/omote","gotou/omote","mimura/omote",
       "arai/omote","isii/omote"]
ura=["mine/ura","okutani/ura","tochikubo/ura",
       "nakamura/ura","gotou/ura","mimura/ura",
       "arai/ura","isii/ura"]
alart=0
target_omote=[]
target_ura=[]
laplace5=[[-1,-3,-4,-3,-1],
         [-3,0,6,0,-3],
         [-4,6,20,6,-4],
         [-3,0,6,0,-3],
         [-1,-3,-4,-3,-1]]
laplace3=[[1,1,1],
         [1,-8,1],
         [1,1,1]]
passpercent=[]
notpasspercent=[]
omotepercent=[]
urapercent=[]
for k in range(8):#変更必要
    for i in range(10):
        target_omote.append("./graytest/"+str(omote[k])+"/"+str(i)+".jpg")#変更必要
        target_ura.append("./graytest/"+str(ura[k])+"/"+str(i)+".jpg")#変更必要


im_rows = 64 #変更必要
im_cols = 64 #変更必要
im_color = 1 #変更必要
in_shape = (im_rows, im_cols, im_color)
nb_classes = 16 #変更必要
#配列の最大値
max_score = 0.0

#ラベル
LABELS=["江波戸の表","江波戸の裏","奥谷君の表","奥谷君の裏",
       "栃窪先生の表","栃窪先生の裏","中村君の表","中村君の裏",
       "後藤さんの表","後藤さんの裏","三村君の表","三村君の裏",
       "新井さんの表","新井さんの裏","石井さんの表","石井さんの裏"]

model = get_model(in_shape, nb_classes)
model.load_weights('./hdf/11-3min/num16karnel5light64pxbatch256.hdf5')#変更必要

def findmaxscore(per):
    global max_score
    if per!=100.0:
        if max_score<=per:
            max_score=per

def check_photo(path):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    karnel=np.array(laplace5,np.float64)#変更必要
    img=cv2.filter2D(img,-1,karnel)
    img=cv2.resize(img,(im_rows,im_cols))
    plt.imshow(img,cmap="gray")
    plt.show()

    x=np.asarray(img)
    x=x.reshape(-1,im_rows,im_cols,im_color)
    x=x/255
    
    #予測
    pre=model.predict([x])[0]#画像の予測
    idx=pre.argmax()
    per=(pre[idx]*100)
    if per<99.999 and ("omote" in path): #閾値変更箇所
        omotepercent.append(per)
    elif per<99.999 and ("ura" in path): #閾値変更箇所
        urapercent.append(per)
    else:
        pass
        
        
    return(idx,per)

def check_photo_str(path,i):
    idx,per=check_photo(path)
    if per>=99.999: #閾値変更箇所
        print("この写真は、",LABELS[idx],"です。")
        #閾値以上なのに識別を間違えているときの処理
        amari=i/10
        if "表" in LABELS[idx]=="表" in LABELS[math.floor(amari)]:
            if LABELS[idx]!=LABELS[math.floor(amari)]:
                alart=1
                print('識別エラーが起きています'+'\033[31m')
        if "裏" in LABELS[idx]=="裏" in LABELS[math.floor(amari)]:
            if LABELS[idx]!=LABELS[math.floor(amari)]:
                alart=1
                print('識別エラーが起きています'+'\033[31m')
        #-------------------------------------------
        print("可能性は、",per,"%です。")
        passpercent.append(per)
        #配列の最大値を見つける
        findmaxscore(per)
    else:
        print('\033[31m'+"可能性が",per,"%なので認証できません。"+'\033[31m')
        print("認証の結果は",LABELS[idx],"でした。")
        notpasspercent.append(per)
    
if __name__=='__main__':
    number=[9,19,29,39,49,59,69,79,89,99,109,119,129,139,149]
    for i in range(80):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_omote[i],i)
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
    for i in range(80):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_ura[i],i)
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
    print(passpercent)
    print(notpasspercent)
    print("閾値以上で識別したテスト数"+str(len(passpercent)))
    print("閾値未満で識別したテスト数"+str(len(notpasspercent)))
    print("表の手だけで閾値未満で識別したテスト数"+str(len(omotepercent)))
    print("裏の手だけで閾値未満で識別したテスト数"+str(len(urapercent)))
    totalpass=len(passpercent)/160
    totalnotpass=len(notpasspercent)/160
    omotenotpass=len(omotepercent)/80
    omotepass=(80-len(omotepercent))/80
    uranotpass=len(urapercent)/80
    urapass=(80-len(urapercent))/80
    print("================================================")
    print("未学習の手を閾値未満で識別した割合"+str(round(totalnotpass*100))+"%")
    print("未学習の手を閾値以上で識別した割合"+str(round(totalpass*100))+"%")
    print("表の手だけで未学習の手を閾値未満で識別した割合"+str(round(omotenotpass*100))+"%")
    print("表の手だけで未学習の手を閾値以上で識別した割合"+str(round(omotepass*100))+"%")
    print("裏の手だけで未学習の手を閾値未満で識別した割合"+str(round(uranotpass*100))+"%")
    print("裏の手だけで未学習の手を閾値以上で識別した割合"+str(round(urapass*100))+"%")
    print("閾値以上の最大値は"+str(max_score)) #11月追加
    print("閾値以上の最小値は"+str(min(passpercent))) #11月追加
    #閾値以上なのに識別を間違えているときのエラーを出力する
    if alart==1:
        print('識別エラーが起きています'+'\033[31m')
